## Introduction to modelling with AleaCarta
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/artefactory/choice-learn/blob/main/notebooks/basket_models/alea_carta.ipynb)

We use a synthetic dataset to demonstrate how to use the AleaCarta model [1]

In [ ]:
# Install necessary packages that are not already installed in the environment
# !pip install matplotlib

In [ ]:
import os
import sys

os.environ["CUDA_VISIBLE_DEVICES"] = ""

sys.path.append("./../../")
print(os.getcwd())

import matplotlib.pyplot as plt
import numpy as np

from choice_learn.basket_models import Trip, TripDataset
from choice_learn.basket_models import AleaCarta


In [ ]:
from synthetic_dataset import get_dataset_ace

data = get_dataset_ace()

print(data)
print(f"\nThe TripDataset 'data' contains {data.n_items} distinct items that appear in {data.n_samples} transactions carried out at {data.n_stores} point of sale with {data.n_assortments} different assortments.")

In [ ]:
latent_sizes = {"preferences": 6, "price": 3, "season": 3}
n_negative_samples = 2
optimizer = "adam"
lr = 1e-2
epochs = 200
# epochs = 1000
batch_size = 32

In [ ]:
model = AleaCarta(
    # item_intercept=True,
    item_intercept=False,
    price_effects=False,
    seasonal_effects=False,
    latent_sizes=latent_sizes,
    n_negative_samples=n_negative_samples,
    optimizer=optimizer,
    lr=lr,
    epochs=epochs,
    batch_size=batch_size,
)

model.instantiate(n_items=7, n_stores=2)

In [ ]:
history = model.fit(trip_dataset=data)

In [ ]:
plt.plot(history["train_loss"])
plt.xlabel("Epoch")
plt.ylabel("Training Loss")
plt.legend()
plt.title("Training of Shopper")
plt.show()

### References
[1] Better Capturing Interactions between Products in Retail: Revisited Negative Sampling for Basket Choice Modeling, Désir, J.; Auriau, V.; Možina, M.; Malherbe, E. (2025), ECML-PKDDD